In [2]:
import cv2
from keras.models import load_model

# Load the cascade classifiers for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to detect faces in an image
def detect_faces(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

# Function to draw rectangles around detected faces
def draw_faces(img, faces):
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

# Main function
def main():
    # Open the camera
    cap = cv2.VideoCapture(0)

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Detect faces
        faces = detect_faces(frame)

        # Draw rectangles around faces
        draw_faces(frame, faces)

        # Display the resulting frame
        cv2.imshow('Face Detection', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [4]:
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from openpyxl import Workbook, load_workbook

def image_to_embedding(img_path, model):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get the embedding vector
    embedding = model.predict(img_array)

    return embedding.flatten()

def main():
    # Load the pre-trained ResNet50 model
    model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    excel_file = 'emotion_data_vector.xlsx'
    
    try:
        wb = load_workbook(excel_file)
        sheet = wb.active
    except FileNotFoundError:
        wb = Workbook()
        sheet = wb.active
        sheet.append(["Shape", "Vector"])
    
    # Path to your image
    img_path = 'shreyash.jpg'

    # Get the embedding vector for the image
    embedding_vector = image_to_embedding(img_path, model)

    sheet.append([str(embedding_vector.shape), str(embedding_vector)])
    wb.save(excel_file)

if __name__ == "__main__":
    main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
